# 🔧 LangChain学习故障排除指南

## 📋 目录
1. [环境配置问题](#1-环境配置问题)
2. [API密钥和认证问题](#2-api密钥和认证问题)
3. [代码执行问题](#3-代码执行问题)
4. [性能和优化问题](#4-性能和优化问题)
5. [学习方法问题](#5-学习方法问题)
6. [诊断工具](#6-诊断工具)

---

In [ ]:
# 导入诊断工具
import sys
import os
import platform
import subprocess
from pathlib import Path
import importlib

print("🔧 LangChain故障排除工具已加载")
print("使用本指南可以解决学习过程中遇到的常见问题")

## 1. 🛠️ 环境配置问题

### 1.1 Python版本检查

In [ ]:
def check_python_version():
    """检查Python版本是否符合要求"""
    print("🐍 Python版本检查")
    print("="*40)
    
    version = sys.version_info
    print(f"当前Python版本: {version.major}.{version.minor}.{version.micro}")
    print(f"完整版本信息: {sys.version}")
    print(f"平台信息: {platform.platform()}")
    
    if version >= (3, 9):
        print("✅ Python版本符合要求 (>= 3.9)")
        return True
    else:
        print("⚠️ Python版本过低，建议升级到3.9或更高版本")
        print("\n升级建议:")
        print("• 使用pyenv: pyenv install 3.11 && pyenv global 3.11")
        print("• 使用conda: conda install python=3.11")
        print("• 下载官方安装包: https://www.python.org/downloads/")
        return False

check_python_version()

### 1.2 包依赖检查

In [ ]:
def check_package_dependencies():
    """检查必需的Python包是否已安装"""
    print("📦 包依赖检查")
    print("="*40)
    
    required_packages = {
        'langchain': '核心框架',
        'langchain_openai': 'OpenAI集成',
        'langchain_core': '核心组件',
        'openai': 'OpenAI客户端',
        'pandas': '数据处理',
        'matplotlib': '数据可视化',
        'numpy': '数值计算',
        'jupyter': 'Jupyter环境',
        'python_dotenv': '环境变量管理',
        'pydantic': '数据验证',
        'requests': 'HTTP请求'
    }
    
    installed_packages = []
    missing_packages = []
    
    for package, description in required_packages.items():
        try:
            module = importlib.import_module(package)
            version = getattr(module, '__version__', 'Unknown')
            print(f"✅ {package:<20} v{version:<10} - {description}")
            installed_packages.append(package)
        except ImportError:
            print(f"❌ {package:<20} {'未安装':<10} - {description}")
            missing_packages.append(package)
    
    print(f"\n📊 统计: {len(installed_packages)}/{len(required_packages)} 已安装")
    
    if missing_packages:
        print(f"\n🔧 修复命令:")
        install_cmd = f"pip install {' '.join(missing_packages)}"
        print(f"  {install_cmd}")
        
        # 提供分组安装命令
        core_packages = [p for p in missing_packages if 'langchain' in p]
        other_packages = [p for p in missing_packages if 'langchain' not in p]
        
        if core_packages:
            print(f"\n核心包: pip install {' '.join(core_packages)}")
        if other_packages:
            print(f"辅助包: pip install {' '.join(other_packages)}")
    else:
        print("\n🎉 所有必需包都已安装！")
    
    return len(missing_packages) == 0

check_package_dependencies()

### 1.3 虚拟环境检查

In [ ]:
def check_virtual_environment():
    """检查是否在虚拟环境中运行"""
    print("🌐 虚拟环境检查")
    print("="*40)
    
    # 检查虚拟环境
    in_venv = hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix)
    
    print(f"Python路径: {sys.executable}")
    print(f"系统前缀: {sys.prefix}")
    
    if in_venv:
        print("✅ 正在虚拟环境中运行")
        
        # 尝试获取虚拟环境名称
        venv_name = os.path.basename(sys.prefix)
        print(f"环境名称: {venv_name}")
        
    else:
        print("⚠️ 未检测到虚拟环境")
        print("\n建议创建虚拟环境:")
        print("• 使用venv: python -m venv langchain_env")
        print("• 激活环境: source langchain_env/bin/activate (Linux/Mac)")
        print("• 激活环境: langchain_env\\Scripts\\activate (Windows)")
        print("• 使用conda: conda create -n langchain python=3.11")
        print("• 激活conda: conda activate langchain")
    
    # 检查VIRTUAL_ENV环境变量
    virtual_env = os.getenv('VIRTUAL_ENV')
    if virtual_env:
        print(f"VIRTUAL_ENV: {virtual_env}")
    
    return in_venv

check_virtual_environment()

## 2. 🔑 API密钥和认证问题

### 2.1 环境变量检查

In [ ]:
def check_api_keys():
    """检查API密钥配置"""
    print("🔑 API密钥检查")
    print("="*40)
    
    # 尝试加载.env文件
    try:
        from dotenv import load_dotenv
        load_dotenv()
        print("✅ .env文件已加载")
    except ImportError:
        print("⚠️ python-dotenv未安装，可能无法加载.env文件")
    except Exception as e:
        print(f"⚠️ 加载.env文件失败: {e}")
    
    # 检查各种API密钥
    api_keys = {
        'OPENAI_API_KEY': 'OpenAI API密钥',
        'ANTHROPIC_API_KEY': 'Anthropic Claude API密钥',
        'LANGCHAIN_API_KEY': 'LangSmith API密钥',
        'LANGCHAIN_TRACING_V2': 'LangSmith追踪',
        'LANGCHAIN_PROJECT': 'LangSmith项目名'
    }
    
    configured_keys = []
    missing_keys = []
    
    for key, description in api_keys.items():
        value = os.getenv(key)
        if value:
            if 'API_KEY' in key:
                # 隐藏API密钥的大部分内容
                masked_value = value[:8] + '*' * (len(value) - 12) + value[-4:] if len(value) > 12 else '*' * len(value)
                print(f"✅ {key:<25} - {description} ({masked_value})")
            else:
                print(f"✅ {key:<25} - {description} ({value})")
            configured_keys.append(key)
        else:
            print(f"❌ {key:<25} - {description} (未配置)")
            missing_keys.append(key)
    
    print(f"\n📊 统计: {len(configured_keys)}/{len(api_keys)} 已配置")
    
    if missing_keys:
        print(f"\n🔧 配置建议:")
        print("1. 创建.env文件在项目根目录")
        print("2. 添加以下内容到.env文件:")
        for key in missing_keys:
            print(f"   {key}=your_{key.lower()}_here")
        print("3. 重启Jupyter内核以加载新的环境变量")
    
    return len(missing_keys) == 0

check_api_keys()

### 2.2 API连接测试

In [ ]:
def test_openai_connection():
    """测试OpenAI API连接"""
    print("🔌 OpenAI API连接测试")
    print("="*40)
    
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        print("❌ OPENAI_API_KEY未配置")
        return False
    
    try:
        # 尝试导入OpenAI
        from openai import OpenAI
        print("✅ OpenAI包导入成功")
        
        # 创建客户端
        client = OpenAI(api_key=api_key)
        print("✅ OpenAI客户端创建成功")
        
        # 测试简单的API调用
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": "测试连接，请回复'连接成功'"}],
                max_tokens=10
            )
            print("✅ API调用成功")
            print(f"响应: {response.choices[0].message.content}")
            return True
            
        except Exception as api_error:
            print(f"❌ API调用失败: {api_error}")
            
            # 分析常见错误
            error_str = str(api_error).lower()
            if "invalid api key" in error_str or "unauthorized" in error_str:
                print("💡 可能的原因: API密钥无效或已过期")
                print("   解决方案: 检查API密钥是否正确，或重新生成")
            elif "insufficient quota" in error_str or "billing" in error_str:
                print("💡 可能的原因: 账户余额不足")
                print("   解决方案: 充值账户或检查使用限制")
            elif "rate limit" in error_str:
                print("💡 可能的原因: 请求频率过高")
                print("   解决方案: 降低请求频率或升级账户")
            else:
                print("💡 建议: 检查网络连接和API密钥")
            return False
            
    except ImportError:
        print("❌ OpenAI包未安装")
        print("🔧 修复命令: pip install openai")
        return False
    except Exception as e:
        print(f"❌ 连接测试失败: {e}")
        return False

def test_langchain_openai():
    """测试LangChain OpenAI集成"""
    print("\n🔗 LangChain OpenAI集成测试")
    print("="*40)
    
    try:
        from langchain_openai import ChatOpenAI
        print("✅ LangChain OpenAI包导入成功")
        
        # 创建LLM实例
        llm = ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0,
            max_tokens=10
        )
        print("✅ ChatOpenAI实例创建成功")
        
        # 测试调用
        try:
            response = llm.invoke("测试LangChain连接")
            print("✅ LangChain调用成功")
            print(f"响应: {response.content}")
            return True
        except Exception as e:
            print(f"❌ LangChain调用失败: {e}")
            return False
            
    except ImportError:
        print("❌ LangChain OpenAI包未安装")
        print("🔧 修复命令: pip install langchain-openai")
        return False

# 运行连接测试
openai_ok = test_openai_connection()
langchain_ok = test_langchain_openai()

print(f"\n📋 连接测试总结:")
print(f"  OpenAI API: {'✅' if openai_ok else '❌'}")
print(f"  LangChain集成: {'✅' if langchain_ok else '❌'}")

if not (openai_ok or langchain_ok):
    print("\n⚠️ 注意: API连接失败，但您仍可以：")
    print("  • 学习理论概念和代码结构")
    print("  • 使用模拟模式运行示例")
    print("  • 配置好API后再进行实际调用")

## 3. 🐛 代码执行问题

### 3.1 常见导入错误

In [ ]:
def diagnose_import_errors():
    """诊断常见的导入错误"""
    print("📥 导入错误诊断")
    print("="*40)
    
    # 测试常见的导入组合
    import_tests = [
        ("langchain", "LangChain核心库"),
        ("langchain_core.prompts", "核心提示词模块"),
        ("langchain_core.output_parsers", "输出解析器"),
        ("langchain_core.tools", "工具系统"),
        ("langchain_openai", "OpenAI集成"),
        ("langchain.agents", "Agent系统"),
        ("langchain.chains", "Chain系统"),
        ("langchain.memory", "记忆系统")
    ]
    
    success_count = 0
    
    for module_name, description in import_tests:
        try:
            importlib.import_module(module_name)
            print(f"✅ {module_name:<30} - {description}")
            success_count += 1
        except ImportError as e:
            print(f"❌ {module_name:<30} - {description}")
            print(f"   错误: {e}")
            
            # 提供修复建议
            if "langchain_openai" in module_name:
                print(f"   修复: pip install langchain-openai")
            elif "langchain" in module_name:
                print(f"   修复: pip install langchain")
    
    print(f"\n📊 导入成功率: {success_count}/{len(import_tests)} ({success_count/len(import_tests)*100:.1f}%)")
    
    if success_count < len(import_tests):
        print("\n🔧 推荐的修复命令:")
        print("pip install --upgrade langchain langchain-openai langchain-core")
    
    return success_count == len(import_tests)

diagnose_import_errors()

### 3.2 内存和性能诊断

In [ ]:
def check_system_resources():
    """检查系统资源状况"""
    print("💻 系统资源检查")
    print("="*40)
    
    try:
        import psutil
        
        # 内存信息
        memory = psutil.virtual_memory()
        print(f"总内存: {memory.total / (1024**3):.1f} GB")
        print(f"可用内存: {memory.available / (1024**3):.1f} GB")
        print(f"内存使用率: {memory.percent:.1f}%")
        
        # CPU信息
        cpu_percent = psutil.cpu_percent(interval=1)
        print(f"CPU使用率: {cpu_percent:.1f}%")
        print(f"CPU核心数: {psutil.cpu_count()}")
        
        # 磁盘信息
        disk = psutil.disk_usage('.')
        print(f"磁盘空间: {disk.free / (1024**3):.1f} GB 可用 / {disk.total / (1024**3):.1f} GB 总计")
        
        # 性能建议
        print("\n💡 性能建议:")
        if memory.percent > 80:
            print("⚠️ 内存使用率较高，建议关闭其他程序")
        if memory.available / (1024**3) < 1:
            print("⚠️ 可用内存不足1GB，可能影响大模型运行")
        if cpu_percent > 80:
            print("⚠️ CPU使用率较高，建议等待其他任务完成")
        if disk.free / (1024**3) < 5:
            print("⚠️ 磁盘空间不足5GB，建议清理空间")
        
        if memory.percent <= 60 and cpu_percent <= 60:
            print("✅ 系统资源充足，适合运行LangChain应用")
            
    except ImportError:
        print("❌ psutil未安装，无法获取详细的系统资源信息")
        print("安装命令: pip install psutil")
        
        # 使用基础方法获取一些信息
        print(f"\n基础信息:")
        print(f"平台: {platform.platform()}")
        print(f"架构: {platform.architecture()}")
        print(f"处理器: {platform.processor() or 'Unknown'}")

check_system_resources()

### 3.3 Jupyter环境诊断

In [ ]:
def check_jupyter_environment():
    """检查Jupyter环境配置"""
    print("📓 Jupyter环境检查")
    print("="*40)
    
    # 检查Jupyter相关包
    jupyter_packages = ['jupyter', 'notebook', 'jupyterlab', 'ipykernel']
    
    for package in jupyter_packages:
        try:
            module = importlib.import_module(package)
            version = getattr(module, '__version__', 'Unknown')
            print(f"✅ {package:<15} v{version}")
        except ImportError:
            print(f"⚠️ {package:<15} 未安装")
    
    # 检查内核信息
    print(f"\n🔬 内核信息:")
    print(f"Python可执行文件: {sys.executable}")
    print(f"当前工作目录: {os.getcwd()}")
    
    # 检查环境变量
    jupyter_vars = ['JUPYTER_CONFIG_DIR', 'JUPYTER_DATA_DIR', 'JUPYTER_RUNTIME_DIR']
    print(f"\n📁 Jupyter环境变量:")
    for var in jupyter_vars:
        value = os.getenv(var)
        if value:
            print(f"  {var}: {value}")
        else:
            print(f"  {var}: 未设置 (使用默认值)")
    
    # 检查扩展
    try:
        import IPython
        print(f"\n🧩 IPython版本: {IPython.__version__}")
        
        # 获取魔法命令
        from IPython import get_ipython
        ip = get_ipython()
        if ip:
            magic_commands = list(ip.magics_manager.magics['line'].keys())
            print(f"可用魔法命令数量: {len(magic_commands)}")
            print(f"常用命令: %time, %timeit, %matplotlib, %load_ext")
    except:
        print("⚠️ 无法获取IPython信息")
    
    # 提供故障排除建议
    print(f"\n🔧 Jupyter故障排除:")
    print("• 内核崩溃: Kernel → Restart & Clear Output")
    print("• 导入错误: 确保在正确的虚拟环境中")
    print("• 慢响应: 检查系统资源，重启内核")
    print("• 扩展问题: jupyter lab clean && jupyter lab build")

check_jupyter_environment()

## 4. ⚡ 性能和优化问题

### 4.1 API调用优化

In [ ]:
def analyze_api_performance():
    """分析API调用性能"""
    print("⚡ API性能分析")
    print("="*40)
    
    # 性能优化建议
    optimization_tips = {
        "模型选择": [
            "gpt-3.5-turbo: 快速、便宜，适合大多数任务",
            "gpt-4: 质量高但较慢较贵，适合复杂任务",
            "gpt-4-turbo: 平衡性能和速度"
        ],
        "参数优化": [
            "temperature: 0-0.3 提高一致性，0.7-1.0 增加创造性",
            "max_tokens: 限制响应长度以节省成本",
            "stream: 启用流式响应改善用户体验"
        ],
        "提示词优化": [
            "清晰简洁: 避免冗余信息",
            "结构化: 使用明确的指令格式",
            "示例驱动: 提供few-shot examples"
        ],
        "缓存策略": [
            "相同查询缓存结果",
            "使用本地缓存减少API调用",
            "批量处理类似请求"
        ]
    }
    
    for category, tips in optimization_tips.items():
        print(f"\n🎯 {category}:")
        for tip in tips:
            print(f"  • {tip}")
    
    # 成本估算工具
    print(f"\n💰 成本估算 (GPT-3.5-turbo):")
    print(f"  输入: $0.0015 / 1K tokens")
    print(f"  输出: $0.002 / 1K tokens")
    print(f"  示例: 1000字对话约0.5美元")
    
    print(f"\n📊 性能基准:")
    print(f"  gpt-3.5-turbo: ~2-5秒响应时间")
    print(f"  gpt-4: ~10-30秒响应时间")
    print(f"  流式响应: 首字节<1秒")

analyze_api_performance()

### 4.2 内存使用优化

In [ ]:
def memory_optimization_guide():
    """内存优化指导"""
    print("🧠 内存使用优化")
    print("="*40)
    
    # 获取当前内存使用
    try:
        import psutil
        process = psutil.Process()
        memory_info = process.memory_info()
        print(f"当前进程内存使用: {memory_info.rss / 1024 / 1024:.1f} MB")
    except:
        print("无法获取内存使用信息")
    
    print(f"\n💡 内存优化策略:")
    
    strategies = {
        "数据处理": [
            "使用生成器而不是列表存储大量数据",
            "及时删除不需要的变量",
            "分批处理大型数据集"
        ],
        "模型使用": [
            "避免同时加载多个大模型",
            "使用较小的模型进行开发测试",
            "及时释放不用的模型实例"
        ],
        "缓存管理": [
            "限制缓存大小",
            "使用LRU缓存策略",
            "定期清理过期缓存"
        ],
        "Jupyter优化": [
            "定期重启内核清理内存",
            "避免在单个cell中处理过多数据",
            "使用%reset魔法命令清理变量"
        ]
    }
    
    for category, tips in strategies.items():
        print(f"\n🔧 {category}:")
        for tip in tips:
            print(f"  • {tip}")
    
    # 内存监控代码示例
    print(f"\n📝 内存监控示例代码:")
    monitoring_code = """
# 监控内存使用
import psutil
import gc

def check_memory():
    process = psutil.Process()
    return process.memory_info().rss / 1024 / 1024

# 使用前后对比
before = check_memory()
# ... 你的代码 ...
after = check_memory()
print(f"内存变化: {after - before:.1f} MB")

# 手动垃圾回收
gc.collect()
"""
    print(monitoring_code)

memory_optimization_guide()

## 5. 📚 学习方法问题

### 5.1 学习进度诊断

In [ ]:
def learning_progress_analysis():
    """分析学习进度和提供建议"""
    print("📈 学习进度分析")
    print("="*40)
    
    # 导入进度追踪器
    try:
        from progress_tracker import ProgressTracker
        tracker = ProgressTracker()
        progress = tracker.get_progress_summary()
        
        print(f"✅ 进度数据加载成功")
        print(f"总学习时间: {progress['total_time_hours']:.1f} 小时")
        print(f"完成课程数: {progress['completed_lessons']}")
        print(f"完成练习数: {progress['completed_exercises']}")
        print(f"平均分数: {progress['average_score']:.1f}")
        
        # 分析学习模式
        print(f"\n🔍 学习模式分析:")
        
        hours = progress['total_time_hours']
        lessons = progress['completed_lessons']
        exercises = progress['completed_exercises']
        score = progress['average_score']
        
        # 学习速度分析
        if hours > 0 and lessons > 0:
            hours_per_lesson = hours / lessons
            print(f"  平均每课程学习时间: {hours_per_lesson:.1f} 小时")
            
            if hours_per_lesson < 0.5:
                print(f"  🚀 学习速度: 较快 - 建议深入练习")
            elif hours_per_lesson < 1.5:
                print(f"  ⚡ 学习速度: 适中 - 保持当前节奏")
            else:
                print(f"  🐌 学习速度: 较慢 - 建议复习基础概念")
        
        # 练习完成度分析
        if lessons > 0:
            exercises_per_lesson = exercises / lessons if lessons > 0 else 0
            print(f"  平均每课程练习数: {exercises_per_lesson:.1f}")
            
            if exercises_per_lesson < 2:
                print(f"  📝 练习度: 偏低 - 建议多做练习")
            elif exercises_per_lesson < 4:
                print(f"  📚 练习度: 适中 - 继续保持")
            else:
                print(f"  🏆 练习度: 很高 - 学习态度优秀")
        
        # 成绩分析
        if score >= 90:
            print(f"  🎯 掌握程度: 优秀 - 可以尝试高级内容")
        elif score >= 75:
            print(f"  👍 掌握程度: 良好 - 继续当前进度")
        elif score >= 60:
            print(f"  📖 掌握程度: 一般 - 建议重点复习")
        else:
            print(f"  ⚠️ 掌握程度: 需提高 - 从基础开始巩固")
    
    except:
        print("⚠️ 无法加载进度数据，提供一般性建议")
    
    # 通用学习建议
    print(f"\n💡 学习方法建议:")
    
    learning_tips = [
        "📚 理论与实践结合: 理解概念后立即动手练习",
        "🔄 定期复习: 每周回顾之前学过的内容",
        "🎯 项目驱动: 通过实际项目应用所学知识",
        "👥 社区参与: 参与讨论，分享学习心得",
        "📝 笔记记录: 记录重要概念和代码片段",
        "🐛 错误分析: 认真分析每个错误的原因",
        "⏰ 时间管理: 每次学习30-60分钟，保持专注"
    ]
    
    for tip in learning_tips:
        print(f"  • {tip}")

learning_progress_analysis()

### 5.2 常见学习困难及解决方案

In [ ]:
def common_learning_challenges():
    """分析常见学习困难并提供解决方案"""
    print("🤔 常见学习困难分析")
    print("="*40)
    
    challenges = {
        "概念理解困难": {
            "症状": ["理论看不懂", "概念混淆", "不知道怎么应用"],
            "原因": ["基础知识不足", "学习顺序错误", "缺少实践"],
            "解决方案": [
                "重新学习Python基础",
                "按照推荐顺序学习",
                "多看示例代码",
                "画概念图帮助理解"
            ]
        },
        "代码实践困难": {
            "症状": ["代码总是出错", "不会修改示例", "无法独立编写"],
            "原因": ["编程基础薄弱", "调试能力不足", "练习不够"],
            "解决方案": [
                "加强Python编程练习",
                "学会使用调试工具",
                "从简单修改开始",
                "逐步增加复杂度"
            ]
        },
        "环境配置困难": {
            "症状": ["包安装失败", "API调用不成功", "环境冲突"],
            "原因": ["网络问题", "权限不足", "版本冲突"],
            "解决方案": [
                "使用虚拟环境",
                "查看详细错误信息",
                "使用国内镜像源",
                "寻求技术支持"
            ]
        },
        "学习动力不足": {
            "症状": ["容易放弃", "进度缓慢", "缺乏兴趣"],
            "原因": ["目标不明确", "没有成就感", "学习孤单"],
            "解决方案": [
                "设定具体的学习目标",
                "庆祝小的进步",
                "找学习伙伴",
                "关注实际应用价值"
            ]
        }
    }
    
    for challenge, details in challenges.items():
        print(f"\n🎯 {challenge}")
        print(f"  症状: {', '.join(details['症状'])}")
        print(f"  原因: {', '.join(details['原因'])}")
        print(f"  解决方案:")
        for solution in details['解决方案']:
            print(f"    • {solution}")
    
    # 自我诊断问卷
    print(f"\n📋 学习困难自我诊断:")
    print("请诚实回答以下问题（是/否）:")
    
    questions = [
        "我经常看不懂理论概念",
        "我的代码经常出现错误",
        "我很难配置开发环境",
        "我经常想要放弃学习",
        "我不知道如何应用所学知识",
        "我觉得学习进度太慢",
        "我缺乏学习的动力"
    ]
    
    for i, question in enumerate(questions, 1):
        print(f"  {i}. {question}")
    
    print(f"\n💡 诊断说明:")
    print(f"  • 1-2个'是': 学习状态良好")
    print(f"  • 3-4个'是': 需要调整学习方法")
    print(f"  • 5+个'是': 建议寻求帮助和指导")

common_learning_challenges()

## 6. 🔍 诊断工具

### 6.1 一键环境诊断

In [ ]:
def comprehensive_diagnosis():
    """综合环境诊断"""
    print("🔍 开始综合环境诊断")
    print("="*60)
    
    diagnosis_results = {}
    
    # 1. Python版本检查
    print("\n1️⃣ Python版本检查")
    diagnosis_results['python'] = check_python_version()
    
    # 2. 包依赖检查
    print("\n2️⃣ 包依赖检查")
    diagnosis_results['packages'] = check_package_dependencies()
    
    # 3. 虚拟环境检查
    print("\n3️⃣ 虚拟环境检查")
    diagnosis_results['venv'] = check_virtual_environment()
    
    # 4. API密钥检查
    print("\n4️⃣ API密钥检查")
    diagnosis_results['api_keys'] = check_api_keys()
    
    # 5. 系统资源检查
    print("\n5️⃣ 系统资源检查")
    check_system_resources()
    diagnosis_results['resources'] = True  # 假设通过
    
    # 6. Jupyter环境检查
    print("\n6️⃣ Jupyter环境检查")
    check_jupyter_environment()
    diagnosis_results['jupyter'] = True  # 假设通过
    
    # 生成诊断报告
    print("\n" + "="*60)
    print("📊 诊断报告总结")
    print("="*60)
    
    passed_count = sum(diagnosis_results.values())
    total_count = len(diagnosis_results)
    
    for check, result in diagnosis_results.items():
        status = "✅ 通过" if result else "❌ 失败"
        print(f"{check:<15}: {status}")
    
    print(f"\n总体状态: {passed_count}/{total_count} 项检查通过")
    
    # 生成建议
    if passed_count == total_count:
        print("\n🎉 恭喜！您的环境配置完美！")
        print("可以开始愉快的LangChain学习之旅了！")
    elif passed_count >= total_count * 0.7:
        print("\n👍 您的环境基本配置正确！")
        print("修复剩余问题后即可获得最佳学习体验。")
    else:
        print("\n⚠️ 您的环境需要一些修复")
        print("建议按照上述检查结果逐项修复问题。")
    
    # 生成修复脚本
    print(f"\n🔧 快速修复脚本:")
    fix_commands = []
    
    if not diagnosis_results.get('packages', True):
        fix_commands.append("pip install langchain langchain-openai pandas matplotlib jupyter python-dotenv")
    
    if not diagnosis_results.get('api_keys', True):
        fix_commands.append("# 创建.env文件并添加API密钥")
    
    if fix_commands:
        for cmd in fix_commands:
            print(f"  {cmd}")
    else:
        print("  无需修复命令！")
    
    return diagnosis_results

# 运行综合诊断
diagnosis = comprehensive_diagnosis()

### 6.2 生成诊断报告

In [ ]:
def generate_diagnosis_report():
    """生成详细的诊断报告"""
    from datetime import datetime
    
    print("📄 生成诊断报告")
    print("="*40)
    
    # 收集系统信息
    report_data = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'platform': platform.platform(),
        'python_version': f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}",
        'python_executable': sys.executable,
        'working_directory': os.getcwd(),
        'environment_variables': {
            'OPENAI_API_KEY': '已配置' if os.getenv('OPENAI_API_KEY') else '未配置',
            'VIRTUAL_ENV': os.getenv('VIRTUAL_ENV', '未使用虚拟环境')
        }
    }
    
    # 生成报告内容
    report_content = f"""
# LangChain学习环境诊断报告

生成时间: {report_data['timestamp']}

## 系统信息
- 操作系统: {report_data['platform']}
- Python版本: {report_data['python_version']}
- Python路径: {report_data['python_executable']}
- 工作目录: {report_data['working_directory']}

## 环境配置
- OpenAI API密钥: {report_data['environment_variables']['OPENAI_API_KEY']}
- 虚拟环境: {report_data['environment_variables']['VIRTUAL_ENV']}

## 建议的下一步行动
1. 如有配置问题，请按照故障排除指南修复
2. 确保所有必需的包都已安装
3. 配置API密钥以获得完整功能
4. 开始系统性的LangChain学习

## 学习资源
- 综合学习指南: comprehensive_guide.ipynb
- 进度追踪: progress_tracker.py
- 学习辅助工具: learning_helpers.py

---
报告由LangChain学习环境自动生成
"""
    
    # 保存报告到文件
    report_filename = f"diagnosis_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
    
    try:
        with open(report_filename, 'w', encoding='utf-8') as f:
            f.write(report_content)
        print(f"✅ 诊断报告已保存: {report_filename}")
    except Exception as e:
        print(f"❌ 保存报告失败: {e}")
    
    # 显示报告摘要
    print(f"\n📋 报告摘要:")
    print(f"  时间: {report_data['timestamp']}")
    print(f"  平台: {report_data['platform']}")
    print(f"  Python: {report_data['python_version']}")
    
    return report_content

# 生成报告
report = generate_diagnosis_report()

print("\n🎉 故障排除指南使用完成！")
print("如果问题仍然存在，请：")
print("  1. 查看生成的诊断报告")
print("  2. 参考comprehensive_guide.ipynb")
print("  3. 寻求社区或技术支持")

## 📞 获取更多帮助

如果本故障排除指南无法解决您的问题，请尝试以下资源：

### 📚 官方文档
- [LangChain官方文档](https://python.langchain.com/)
- [OpenAI API文档](https://platform.openai.com/docs)
- [Jupyter文档](https://jupyter.org/documentation)

### 💬 社区支持
- [LangChain GitHub](https://github.com/langchain-ai/langchain)
- [Stack Overflow](https://stackoverflow.com/questions/tagged/langchain)
- [Reddit r/MachineLearning](https://www.reddit.com/r/MachineLearning/)

### 🎓 学习资源
- [DeepLearning.AI课程](https://learn.deeplearning.ai/)
- [LangChain YouTube频道](https://www.youtube.com/@LangChain)
- [AI相关博客和教程](https://blog.langchain.dev/)

### 🛠️ 开发工具
- [LangSmith调试平台](https://smith.langchain.com/)
- [VS Code Python扩展](https://marketplace.visualstudio.com/items?itemName=ms-python.python)
- [GitHub Copilot](https://github.com/features/copilot)

---

**记住：遇到问题是学习过程的正常部分，保持耐心和学习的热情！** 🌟